In this section we will investigate and Compare different join strategies by joining the fact_int_sales and dim_currency
- First we Will cash the two tables, since this notebook will be part of and ETL Job so these tables and joins will be reused, hence cashing is a good idea.

In [0]:
#Load Data
options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": ";"
} 
dim_product =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_product/")
fact_int_sales =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/fact_int_sales/")
dim_sales_terr =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_sales_terr/")
dim_curreny = spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_curreny/")  

In [0]:
display(dim_curreny.limit(10))

CurrencyKey,CurrencyAlternateKey,CurrencyName
1,AFA,Afghani
2,DZD,Algerian Dinar
3,ARS,Argentine Peso
4,AMD,Armenian Dram
5,AWG,Aruban Guilder
6,AUD,Australian Dollar
7,AZM,Azerbaijanian Manat
8,BSD,Bahamian Dollar
9,BHD,Bahraini Dinar
10,THB,Baht


In [0]:
display(fact_int_sales.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06
344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06
312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07


In [0]:
#fact_sales = fact_int_sales.cache()
#dim_curreny = dim_curreny.cache()
#fact_sales.count()  # Force caching (Since Cashing is a lazy evaluatioin)
#dim_curreny.count() 
join_key = "CurrencyKey"

when Joining
    - Spark reads data in partitions, and when performing joins, it shuffles data to ensure that rows with matching keys are placed on the same executor.
    - **Example**: Given `fact_int_sales` and `dim_currency` tables, both with a `CurrencyKey` column, Spark distributes data across executors. After reading, Spark shuffles the data to match rows with the same `ProductKey` for the join.

## Broadcast Join

In [0]:
from pyspark.sql.functions import broadcast

broadcast_join = fact_int_sales.join(
    broadcast(dim_curreny), 
    join_key,
    how="left"
)


In [0]:
display(broadcast_join.limit(10))

CurrencyKey,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,CurrencyAlternateKey,CurrencyName
19,310,20101229,20110110,20110105,21768,1,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05,CAD,Canadian Dollar
39,346,20101229,20110110,20110105,28389,1,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,FRF,French Franc
100,346,20101229,20110110,20110105,25863,1,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,USD,US Dollar
100,336,20101229,20110110,20110105,14501,1,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05,USD,US Dollar
6,346,20101229,20110110,20110105,11003,1,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,AUD,Australian Dollar
100,311,20101230,20110111,20110106,27645,1,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,USD,US Dollar
6,310,20101230,20110111,20110106,16624,1,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,351,20101230,20110111,20110106,11005,1,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,344,20101230,20110111,20110106,11011,1,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
100,312,20101231,20110112,20110107,27621,1,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07,USD,US Dollar


## Sort-Merge Join 

In [0]:
sort_merge_join = fact_int_sales.join(
    dim_curreny, 
    join_key,
    how="left"
)

In [0]:
display(sort_merge_join.limit(10)) 

CurrencyKey,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,CurrencyAlternateKey,CurrencyName
19,310,20101229,20110110,20110105,21768,1,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05,CAD,Canadian Dollar
39,346,20101229,20110110,20110105,28389,1,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,FRF,French Franc
100,346,20101229,20110110,20110105,25863,1,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,USD,US Dollar
100,336,20101229,20110110,20110105,14501,1,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05,USD,US Dollar
6,346,20101229,20110110,20110105,11003,1,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,AUD,Australian Dollar
100,311,20101230,20110111,20110106,27645,1,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,USD,US Dollar
6,310,20101230,20110111,20110106,16624,1,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,351,20101230,20110111,20110106,11005,1,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,344,20101230,20110111,20110106,11011,1,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
100,312,20101231,20110112,20110107,27621,1,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07,USD,US Dollar


## Bucketed Join

In [0]:
# Bucket tables first (if not already)
fact_sales_bucketed = fact_int_sales.repartitionByRange(100, join_key)
dim_curreny_bucketed = dim_curreny.repartitionByRange(100, join_key)

bucketed_join = fact_sales_bucketed.join(
    dim_curreny_bucketed, 
    join_key,
    how="left"
)

In [0]:
display(bucketed_join.limit(10))

CurrencyKey,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,CurrencyAlternateKey,CurrencyName
6,346,20101229,20110110,20110105,11003,1,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,AUD,Australian Dollar
6,310,20101230,20110111,20110106,16624,1,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,351,20101230,20110111,20110106,11005,1,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,344,20101230,20110111,20110106,11011,1,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06,AUD,Australian Dollar
6,313,20101231,20110112,20110107,16351,1,9,SO43709,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07,AUD,Australian Dollar
6,314,20101231,20110112,20110107,16517,1,9,SO43710,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07,AUD,Australian Dollar
6,314,20110102,20110114,20110109,16483,1,9,SO43715,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2011-01-02,2011-01-14,2011-01-09,AUD,Australian Dollar
6,311,20110102,20110114,20110109,16529,1,9,SO43716,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2011-01-02,2011-01-14,2011-01-09,AUD,Australian Dollar
6,336,20110102,20110114,20110109,25249,1,9,SO43717,1,null,null,413.15,413.15,699.1,55.93,174.775,2011-01-02,2011-01-14,2011-01-09,AUD,Australian Dollar
6,311,20110104,20110116,20110111,16520,1,9,SO43724,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2011-01-04,2011-01-16,2011-01-11,AUD,Australian Dollar
